Из курса [Даниила Анастасьева](https://github.com/DanAnastasyev) и [DL School](https://www.dls.samcs.ru)

# General Conversation


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Данные для русского

* Toloka Persona Chat Rus: https://toloka.yandex.ru/datasets/
* Диалоги из литературы: https://github.com/Koziev/NLP_Datasets/blob/master/Conversations/Data/dialogues.zip
* Open Subtitles:http://opus.nlpl.eu/OpenSubtitles-v2018.php

Будем работать с датасетом Толоки

In [ ]:
!rm -f TlkPersonaChatRus.zip
!rm -rf TlkPersonaChatRus
!wget https://tlk.s3.yandex.net/dataset/TlkPersonaChatRus.zip
!unzip TlkPersonaChatRus.zip
!head -n 5 TlkPersonaChatRus/dialogues.tsv

--2022-12-16 14:29:25--  https://tlk.s3.yandex.net/dataset/TlkPersonaChatRus.zip
Resolving tlk.s3.yandex.net (tlk.s3.yandex.net)... 93.158.134.158, 2a02:6b8::2:158
Connecting to tlk.s3.yandex.net (tlk.s3.yandex.net)|93.158.134.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6139432 (5.9M) [application/zip]
Saving to: ‘TlkPersonaChatRus.zip’

TlkPersonaChatRus.z 100%[===================>]   5.85M  10.4MB/s    in 0.6s    

2022-12-16 14:29:26 (10.4 MB/s) - ‘TlkPersonaChatRus.zip’ saved [6139432/6139432]

Archive:  TlkPersonaChatRus.zip
   creating: TlkPersonaChatRus/
  inflating: TlkPersonaChatRus/dialogues.tsv  
  inflating: TlkPersonaChatRus/readme_TlkPersonaChatRus.txt  
  inflating: TlkPersonaChatRus/profiles.tsv  
persona_1_profile	persona_2_profile	dialogue
"<span class=participant_1>У меня любимая работа.<br />Я уважаю людей.<br />У меня есть животное.<br />У меня хороший друг.<br />Я люблю кофе.<br /></span>"	"<span class=participant_2>Ищу принца.<br

Парсим HTML, схлопываем подряд идущие реплики одного человека, убираем пометки о номере пользователя

In [ ]:
import csv
import copy
from html.parser import HTMLParser

class DialogueParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.lines = []
        self.current_line = ""
        self.is_line = False

    def handle_starttag(self, tag, attrs):
        if tag == "span":
            self.is_line = True

    def handle_endtag(self, tag):
        if tag == "span":
            self.lines.append(self.current_line)
            self.is_line = False
            self.current_line = ""

    def handle_data(self, data):
        if self.is_line:
            self.current_line += data
    
    def pop_dialogue(self):
        dialogue = copy.copy(self.lines)
        self.lines = []
        return dialogue


dialogues = []
parser = DialogueParser()
with open("TlkPersonaChatRus/dialogues.tsv", "r") as r:
    next(r)
    reader = csv.reader(r, delimiter='\t')
    for row in reader:
        dialogue = row[2]
        dialogue = dialogue.replace("<br />", " ").replace("<br/>", " ")
        parser.feed(dialogue)
        dialogue = parser.pop_dialogue()
        if not dialogue:
            continue
        user1_start = "Пользователь 1: "
        user2_start = "Пользователь 2: "
        for line in dialogue:
            assert line.startswith(user1_start) or line.startswith(user2_start)

        def get_user(line):
            return 1 if line.startswith(user1_start) else 2
        def clean_line(line):
            return line.replace(user1_start, "").replace(user2_start, "").replace("\n", " ").strip()

        new_dialogue = []
        current_user = get_user(dialogue[0])
        current_line = clean_line(dialogue[0])
        for line in dialogue[1:]:
            user = get_user(line)
            line = clean_line(line)
            if current_user == user:
                current_line += " " + line
                continue
            new_dialogue.append(current_line)
            current_line = line
            current_user = user
        new_dialogue.append(current_line)
        dialogues.append(new_dialogue)

In [ ]:
len(dialogues)

10013

In [ ]:
dialogues[100]

['Привет, друг мой! Как дела, как жизнь? Знакомиться будем?',
 'добрый вечер! конечно, будем, это легально )) как юрист заявляю вам как у вас дела?',
 'Ого! Юрист! Наверное столичный?))) Да вот все супер! Наворачиваю блины с икрой, радуюсь жизни',
 'есть такое) мой папа бизнесмен, а кто то же должен знать юридические тонкости. а кем ты работаешь? мммм блины! а ананасы любишь? я вот обожаю',
 'Завидую белой завистью! А то я сам из провинции, пора бы тоже в столицу! Работа, кстати, у меня интересная .. я биолог .. правда в школе, с сорванцами эими этими* да, ананасы это круто !',
 'учитель! как интересно :) в москве хорошо, любовь вокруг! приезжай',
 'Да с радостью бы! да вот трех котов перевозить- жуть думаю не справлюсь',
 'ого у меня тоже есть кошка! я верю в тебя, справишься ) будем ждать! а я побежал решать свои юридические вопросы) спокойной ночи',
 'пойду тогда почитаю! пока !']

Делаем примеры, состоящие из контекста и ответа

In [ ]:
blocks = []
max_context_length = 1
max_answer_length = 100
for dialogue in dialogues:
    for i in range(1, len(dialogue)):
        context = tuple(dialogue[max(0, i-max_context_length):i])
        answer = dialogue[i]
        blocks.append({"context": context, "answer": answer})
blocks = [block for block in blocks if len(block["answer"]) < max_answer_length]

In [ ]:
len(blocks)

138197

Разбиваем на обучающую и тестовую выборки

In [ ]:
import random
random.shuffle(blocks)
test_part = 0.9
border = int(len(blocks) * test_part)
train_blocks = blocks[:border]
test_blocks = blocks[border:]

BatchIterator разбивает примеры на батчи

In [ ]:
import random
import math
import torch
import numpy as np


class BatchIterator():
    def __init__(self, blocks, batch_size, shuffle=True):
        self.blocks = blocks
        self.num_samples = len(blocks)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.batches_count = int(math.ceil(len(blocks) / batch_size))
        
    def __len__(self):
        return self.batches_count
    
    def __iter__(self):
        indices = np.arange(self.num_samples)
        if self.shuffle:
            np.random.shuffle(indices)

        for start in range(0, self.num_samples, self.batch_size):
            end = min(start + self.batch_size, self.num_samples)
            batch_indices = indices[start:end]
            pivots = []
            positives = []
            for data_ind in batch_indices:
                block = self.blocks[data_ind]
                
                pivots.append(block["context"])
                positives.append(block["answer"])

            yield {
                'pivot_lines': pivots,
                'positive_lines': positives
            }

А дальше у нас есть 2 базовых варианта: ранжирующая и порождающая модели

## Ранжирующая модель

Начнём с ранжирующей модели, она во многом проще в реализации. Нужно заранее составить большую базу ответов и просто выбирать наиболее подходящий к контексту каждый раз. 

![](https://habrastorage.org/web/c79/942/608/c79942608160404ab398033e97283c51.jpg)

*From [Neural conversational models: как научить нейронную сеть светской беседе. Лекция в Яндексе](https://habr.com/ru/company/yandex/blog/333912/)*

Сеть состоит из пары башен: левая кодирует контекст, правая - ответ. Задача - научиться считать близость между представлениями контекста и ответа.



In [ ]:
import torch
import torch.nn as nn

In [ ]:
train_iter = BatchIterator(train_blocks, 8)
test_iter = BatchIterator(test_blocks, 8)

In [ ]:
for batch in train_iter:
    break

In [ ]:
batch['pivot_lines'][:3], batch['positive_lines'][:3]

([('Ой классно как) это очень замечательно',),
  ('А порода какая?',),
  ('Праге нет увы а чем вы любите заниматься в свободное время',)],
 ['Ага 🙈☺️',
  'Сиамская',
  'Люблю смотреть мультики и играть со своими тремя котами'])

### Triplet Loss

Мы хотим не просто научить энкодер строить эмбеддинги для предложений. Мы хотим, чтобы притягивать векторы правильных ответов к вопросам и отталкивать неправильные. Для этого используют, например, *Triplet Loss*:

$$ L = \frac 1N \underset {q, a^+, a^-} \sum max(0, \space \delta - sim[V_q(q), V_a(a^+)] + sim[V_q(q), V_a(a^-)] ),$$

где
* $sim[a, b]$ функция похожести (например, dot product или cosine similarity)
* $\delta$ - гиперпараметр модели. Если $sim[a, b]$ линейно по $b$, то все $\delta > 0$ эквиватентны.

![img](https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/margin.png)

### Hard-negatives mining

Берём в качестве отрицательного примера самый близкий из неправильных примеров в батче:
$$a^-_{hard} = \underset {a^-} {argmax} \space sim[V_q(q), V_a(a^-)]$$

Неправильные в данном случае - все, кроме правильного :)

Реализуется это как-то так:
* Батч состоит из правильных пар.
* Для всех контекстов и всех ответов считаем эмбеддинги.
* Положительные примеры у нас есть - осталось найти для каждого контекста наиболее похожие на него ответы, которые предназначались другим контекстам.

### Задание 1. Реализовать функцию для кодирования предложений

Будем получать векторные представления с помощью модели RuBERT https://huggingface.co/DeepPavlov/distilrubert-base-cased-conversational 

(Если не будет помещаться в памяти, в репозитории DeepPavlov можно взять small или tyni версии модели)

Векторные представления последовательности можно получить двумя способами:
* Усреднив векторные представления токенов
* Получив векторное представление для [CLS] токена.

In [ ]:
from transformers import AutoModel, AutoTokenizer

In [ ]:
DEVICE = torch.device('cuda')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/distilrubert-base-cased-conversational')
bert_left_model = AutoModel.from_pretrained('DeepPavlov/distilrubert-base-cased-conversational').to(DEVICE)
bert_right_model = AutoModel.from_pretrained('DeepPavlov/distilrubert-base-cased-conversational').to(DEVICE)

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/538 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/distilrubert-base-cased-conversational were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at DeepPavlov/distilrubert-base-cased-conversational were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vo

In [ ]:
def get_embedding(model, tokenizer, sentences):
  encodings = tokenizer.batch_encode_plus(sentences, padding=True, return_tensors='pt').to(DEVICE)
  model_states = model(**encodings)
  sentence_embeddings = torch.mean(model_states.last_hidden_state, 1)
  return sentence_embeddings

In [ ]:
class DSSMTripletLoss(nn.Module):
    def __init__(self, left_model, right_model, tokenizer, embedding_dim=300, hidden_dim=300, margin=0.1):
        super().__init__()

        self.left_model = left_model
        self.right_model = right_model
        self.tokenizer = tokenizer

        self.similarity = nn.CosineSimilarity(dim=1)
        self.margin = margin

    def apply(self, pivot_lines, positive_lines, hard_negatives_part=0.0):
        pivots = get_embedding(self.left_model, self.tokenizer, pivot_lines,)
        positives = get_embedding(self.right_model, self.tokenizer, positive_lines,)

        batch_size = pivots.size(0)
        shift = random.randint(1, max(batch_size - 1, 1))

        negative_indices = torch.LongTensor([(i + shift) % batch_size for i in range(batch_size)])
        negatives = positives[negative_indices]

        return pivots, positives, negatives
    
    def calc_recall_at_1(self, pivots, positives, negatives):
        batch_size = pivots.size(0)

        scores = pivots.matmul(positives.transpose(0, 1))
        predicted_indices = torch.argmax(scores, dim=1)

        true_indices = torch.linspace(0, batch_size-1, steps=batch_size)
        correct_count = torch.sum(predicted_indices.cpu() == true_indices).item()
        return correct_count

    def forward(self, pivots, positives, negatives):

        distance = -self.similarity(pivots, positives) + self.similarity(pivots, negatives) + self.margin
        loss = torch.mean(torch.max(distance, torch.zeros_like(distance)))

        return loss
    
    def left_apply(self, lines):
        return get_embedding(self.left_model, self.tokenizer, lines,)
  
    def right_apply(self, lines):
        return get_embedding(self.right_model, self.tokenizer, lines,)

In [ ]:
class ModelTrainer():
    def __init__(self, model, optimizer):
        self.model = model
        self.optimizer = optimizer
        
    def on_epoch_begin(self, is_train, name, batches_count):
        self.epoch_loss = 0
        self.correct_count = 0
        self.total_count = 0
        self.is_train = is_train
        self.name = name
        self.batches_count = batches_count
        self.model.train(is_train)
        
    def on_epoch_end(self):
        return '{:>5s} Loss = {:.5f}, Recall@1 = {:.2%}'.format(
            self.name, self.epoch_loss / self.batches_count, self.correct_count / self.total_count
        )
        
    def on_batch(self, batch):
        pivot_lines = batch['pivot_lines']
        positive_lines = batch['positive_lines']
        pivot_lines = [" ".join(context) for context in pivot_lines]
        #loss
        pivots, positives, negatives = self.model.apply(pivot_lines, positive_lines)
        loss = self.model(pivots, positives, negatives)
        #predicts
        self.correct_count += self.model.calc_recall_at_1(pivots, positives, negatives)
        self.total_count += len(pivot_lines)
        self.epoch_loss += loss.item()
        
        if self.is_train:
            self.optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), 1.)
            self.optimizer.step()

        return '{:>5s} Loss = {:.5f}, Recall@1 = {:.2%}'.format(
            self.name, loss.item(), self.correct_count / self.total_count
        )

In [ ]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(trainer, data_iter, is_train, name=None):
    trainer.on_epoch_begin(is_train, name, batches_count=len(data_iter))
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=len(data_iter)) as progress_bar:
            for i, batch in enumerate(data_iter):
                batch_progress = trainer.on_batch(batch)

                progress_bar.update()
                progress_bar.set_description(batch_progress)
                
            epoch_progress = trainer.on_epoch_end()
            progress_bar.set_description(epoch_progress)
            progress_bar.refresh()

            
def fit(trainer, train_iter, epochs_count=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        do_epoch(trainer, train_iter, is_train=True, name=name_prefix + 'Train:')
        
        if val_iter is not None:
            do_epoch(trainer, val_iter, is_train=False, name=name_prefix + '  Val:')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
import torch.optim as optim
model = DSSMTripletLoss(bert_left_model, bert_right_model, tokenizer).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=2e-5)
trainer = ModelTrainer(model, optimizer)

In [ ]:
fit(trainer, train_iter, epochs_count=2, val_iter=test_iter)

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/left_right_model')

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/left_right_model'))

<All keys matched successfully>

### Задание 2. Реализовать обработку произвольных запросов
* Получить векторные представления для всех примеров из датасета
* В цикле принимать запросы пользователя и возвращать наиболее подходящий ответ

In [ ]:
all_iter = BatchIterator(blocks, 16)
all_emb = []
answers = []

for batch in tqdm(all_iter, total=len(all_iter)):
  pivot_lines = batch['pivot_lines']
  pivot_lines = [' '.join(context) for context in pivot_lines]
  positive_lines = batch['positive_lines']

  curr = []
  curr.extend(pivot_lines)
  curr.extend(positive_lines)
  answers.extend(curr)

  with torch.no_grad():
    embeddings = model.right_apply(curr)
    all_emb.append(embeddings)

100%|██████████| 8638/8638 [04:23<00:00, 32.78it/s]


In [ ]:
answers_embeddings = torch.cat(all_emb)
answers_embeddings = answers_embeddings.to(DEVICE)

In [ ]:
tokenizer.vocab

In [ ]:
import torch.nn.functional as F

similarity = nn.CosineSimilarity(dim=1)
conversation = []

while True:
    sentence = input()
    
    embeddings = model.left_apply([sentence])
    sim = similarity(embeddings, answers_embeddings)
    answer_sentence = answers[torch.argmax(sim)]

    conversation.extend([sentence, answer_sentence])
    print(answer_sentence)

Привет, как дела?
Не плохо, но я скучаю по маме, я люблю её. Как у тебя дела?
Хорошо
Чем ты занимаешься? Помимо работы
Учусь
Где учишься и на кого ?
на хирурга
Круто, хорошие врачи сейчас на вес золота
Привет, как дела?
Не плохо, но я скучаю по маме, я люблю её. Как у тебя дела?


KeyboardInterrupt: ignored

In [ ]:
conversation

# Привет, как дела?
# Хорошо.как твои?
# Отлично, как тебя зовут?
# Артем А тебя?
# Меня зову Коля, чем занимаешься?
# Приятно познакомиться Чем увлекаешься? Я например работаю в кафе, а ты?
# Я сейчас учусь в университете. Как тебе погода за окном?
# А я закончила ин. яз, хорошо владею несколькими языками ))) Взаимно!
# У тебя есть домашние животные?
# Пока нет, а у тебя?
# Тоже нет, но я бы хотел завести собаку.
# Аааа... а родители где?
# Далеко, в другом городе. А ты с родителями живёшь?
# Да, а ты? Я живу в Москве, а ты откуда?
# Я живу один. Сам я из Санкт-Петербурга.
# ООО) интересно. А я из Ватикана? Знаете где это?
# Нет, не бывал там)
# А ну Ок)

['Привет, как дела?',
 'Не плохо, но я скучаю по маме, я люблю её. Как у тебя дела?',
 'Отлично, как тебя зовут?',
 'Александр А тебя как зовут?',
 'Меня зову Коля, чем занимаешься?',
 'Общаюсь с тобой и смотрю сериал. Обожаю смотреть сериалы ! А ты чем занимаешься ?))',
 'Я сейчас учусь в университете. Как тебе погода за окном?',
 'Да погода не очень, я люблю в школе сидеть Ты школьник или в универе уже?',
 'Уже в универе',
 'На кого училась?',
 'Хирург',
 'Ого вот здорово !)хирург ?',
 'Он самый',
 'Он не настоящий какой то',
 'Почему ты так думаешь?',
 'Наверное потому что у меня нет друзей и я одинока',
 'Хочешь я буду твоим другом?',
 'Очень хотелось бы, но не могу - учёба']